In [21]:
import torch, math, re
import numpy as np, pandas as pd
from transformers import AutoTokenizer, AutoModelForMaskedLM

# -------------------- Helper -------------------- #
def get_token_index_from_sequence(input_ids_cpu, special_ids, residue_pos):
    """
    Map 1-based residue_pos → token index by skipping special tokens.
    input_ids_cpu: 1D torch.LongTensor on CPU.
    special_ids: set of token IDs treated as 'special' (CLS, SEP, PAD, etc).
    """
    seq_counter = 0
    for idx, tid in enumerate(input_ids_cpu.tolist()):
        if tid not in special_ids:
            seq_counter += 1
            if seq_counter == residue_pos:
                return idx
    raise ValueError(f"Couldn't map residue {residue_pos} to a token index")

# -------------------- HeatmapGenerator -------------------- #
class HeatmapGenerator:
    def __init__(self, model, tokenizer):
        # pick device
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model  = model.to(self.device)
        self.tokenizer = tokenizer
        self.special_ids = set(tokenizer.all_special_ids)
        self.mask_id = tokenizer.mask_token_id
        self.model.eval()

    def _get_logits(self, ids, mask=None):
        # ids, mask are already on self.device
        with torch.no_grad():
            if mask is not None:
                return self.model(ids, attention_mask=mask).logits
            else:
                return self.model(ids).logits

    def llrData(self, sequence, start_pos=1, end_pos=None):
        if end_pos is None:
            end_pos = len(sequence)
        aas = list("ACDEFGHIKLMNPQRSTVWY")
        heatmap = np.zeros((20, end_pos - start_pos + 1), dtype=float)

        # 1) Tokenize on CPU
        enc_cpu = self.tokenizer(sequence, return_tensors="pt")
        input_ids_cpu = enc_cpu["input_ids"][0]                       # 1D CPU
        attention_cpu = enc_cpu.get("attention_mask", None)          # 2D CPU or None

        # 2) Move to device
        input_ids = enc_cpu["input_ids"].to(self.device)             # [1, L] GPU/CPU
        attention_mask = (attention_cpu.to(self.device)
                          if attention_cpu is not None else None)

        for col, pos in enumerate(range(start_pos, end_pos+1)):
            # a) map residue → token index
            tok_i = get_token_index_from_sequence(input_ids_cpu,
                                                  self.special_ids,
                                                  pos)
            # b) wild‑type token ID BEFORE masking
            wt_id = input_ids[0, tok_i].item()

            # c) mask and forward
            masked = input_ids.clone()
            masked[0, tok_i] = self.mask_id

            logits = self._get_logits(masked, attention_mask)
            log_probs = torch.log_softmax(logits[0, tok_i], dim=-1)

            lp_wt = log_probs[wt_id].item()
            for i, aa in enumerate(aas):
                aa_id = self.tokenizer.convert_tokens_to_ids(aa)
                hm_val = (log_probs[aa_id].item() - lp_wt
                          if aa_id is not None and aa_id < log_probs.size(0)
                          else 0.0)
                heatmap[i, col] = hm_val

        cols = [str(p) for p in range(start_pos, end_pos+1)]
        return pd.DataFrame(heatmap, index=aas, columns=cols)

# -------------------- Single‑mutation Function -------------------- #
def compute_mutation_llr(model, tokenizer, sequence, mutation, device):
    wt, pos, mut = re.match(r"([A-Z])(\d+)([A-Z])", mutation).groups()
    pos = int(pos)
    if sequence[pos-1] != wt:
        print(f"[!] expected '{wt}' at {pos}, found '{sequence[pos-1]}'")

    # 1) Tokenize on CPU → for mapping
    enc_cpu = tokenizer(sequence, return_tensors="pt")
    input_ids_cpu = enc_cpu["input_ids"][0]
    special_ids = set(tokenizer.all_special_ids)
    tok_i = get_token_index_from_sequence(input_ids_cpu, special_ids, pos)

    # 2) Move inputs to device
    enc = {k: v.to(device) for k, v in enc_cpu.items()}
    input_ids = enc["input_ids"]
    attention_mask = enc.get("attention_mask", None)

    # 3) Mask & forward
    masked = input_ids.clone()
    masked[0, tok_i] = tokenizer.mask_token_id

    model.to(device).eval()
    with torch.no_grad():
        out = (model(masked, attention_mask=attention_mask)
               if attention_mask is not None else model(masked))
        log_probs = torch.log_softmax(out.logits[0, tok_i], dim=-1)

    wt_id  = tokenizer.convert_tokens_to_ids(wt)
    mut_id = tokenizer.convert_tokens_to_ids(mut)
    lp_wt  = log_probs[wt_id].item()
    lp_mut = log_probs[mut_id].item()
    return lp_mut - lp_wt, lp_wt, lp_mut

# -------------------- Comparison Runner -------------------- #
def compare_llr_methods(model, tokenizer, sequence, mutations):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    generator = HeatmapGenerator(model, tokenizer)

    print("="*60)
    for mut in mutations:
        llr_fn, lp_wt, lp_mut = compute_mutation_llr(
            model, tokenizer, sequence, mut, device
        )
        pos = int(re.match(r".(\d+).", mut).group(1))
        df = generator.llrData(sequence, start_pos=pos, end_pos=pos)
        llr_hm = df.loc[mut[-1], str(pos)]

        print(f"Mutation {mut}:")
        print(f"  [Function]     LLR: {llr_fn:.6f}, wt: {lp_wt:.6f}, mut: {lp_mut:.6f}")
        print(f"  [HeatmapData]  LLR: {llr_hm:.6f}")
        print(f"  ΔLLR = {abs(llr_fn - llr_hm):.6f}")
        print("-"*60)

In [22]:
# Import packages
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM, AutoModelForMaskedLM
from peft import PeftModel, PeftConfig
from autoamp.evolveFinetune import *
import torch
from tqdm import tqdm
import math
from Bio import SeqIO 
import json
import warnings
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from transformers import PreTrainedTokenizer

# Example inputs
base_model_name = "facebook/esm2_t30_150M_UR50D" 
tokenizer = AutoTokenizer.from_pretrained(base_model_name, use_fast=True)

adapter_checkpoint = "/home/sdowell/scratch/Thesis/BenchmarkingFinetuning/runs/esm_150m_ecoli_finetuning_1/checkpoint-19000"

# Load models
model_pretrained = AutoModelForMaskedLM.from_pretrained(base_model_name)
model_with_adapter = AutoModelForMaskedLM.from_pretrained(base_model_name)
model_finetuned = PeftModel.from_pretrained(model_with_adapter, adapter_checkpoint)

# The protein sequence (concatenated)
topA_seq = (
"MGKALVIVESPAKAKTINKYLGSDYVVKSSVGHIRDLPTSGSAAKKSADSTSTKTAKKPK" 
    "KDERGALVNRMGVDPWHNWEAHYEVLPGKEKVVSELKQLAEKADHIYLATDLDREGEAIA" 
    "WHLREVIGGDDARYSRVVFNEITKNAIRQAFNKPGELNIDRVNAQQARRFMDRVVGYMVS" 
    "PLLWKKIARGLSAGRVQSVAVRLVVEREREIKAFVPEEFWEVDASTTTPSGEALALQVTH" 
    "QNDKPFRPVNKEQTQAAVSLLEKARYSVLEREDKPTTSKPGAPFITSTLQQAASTRLGFG" 
    "VKKTMMMAQRLYEAGYITYMRTDSTNLSQDAVNMVRGYISDNFGKKYLPESPNQYASKEN" 
    "SQEAHEAIRPSDVNVMAESLKDMEADAQKLYQLIWRQFVACQMTPAKYDSTTLTVGAGDF" 
    "RLKARGRILRFDGWTKVMPALRKGDEDRILPAVNKGDALTLVELTPAQHFTKPPARFSEA" 
    "SLVKELEKRGIGRPSTYASIISTIQDRGYVRVENRRFYAEKMGEIVTDRLEENFRELMNY" 
    "DFTAQMENSLDQVANHEAEWKAVLDHFFSDFTQQLDKAEKDPEEGGMRPNQMVLTSIDCP" 
    "TCGRKMGIRTASTGVFLGCSGYALPPKERCKTTINLVPENEVLNVLEGEDAETNALRAKR" 
    "RCPKCGTAMDSYLIDPKRKLHVCGNNPTCDGYEIEEGEFRIKGYDGPIVECEKCGSEMHL" 
    "KMGRFGKYMACTNEECKNTRKILRNGEVAPPKEDPVPLPELPCEKSDAYFVLRDGAAGVF" 
    "LAANTFPKSRETRAPLVEELYRFRDRLPEKLRYLADAPQQDPEGNKTMVRFSRKTKQQYV" 
    "SSEKDGKATGWSAFYVDGKWVEGKK" 
)
spoT_seq = ("MYLFESLNQLIQTYLPEDQIKRLRQAYLVARDAHEGQTRSSGEPYITHPVAVACILAEMK"
                    "LDYETLMAALLHDVIEDTPATYQDMEQLFGKSVAELVEGVSKLDKLKFRDKKEAQAENFR"
                    "KMIMAMVQDIRVILIKLADRTHNMRTLGSLRPDKRRRIARETLEIYSPLAHRLGIHHIKT"
                    "ELEELGFEALYPNRYRVIKEVVKAARGNRKEMIQKILSEIEGRLQEAGIPCRVSGREKHL"
                    "YSIYCKMVLKEQRFHSIMDIYAFRVIVNDSDTCYRVLGQMHSLYKPRPGRVKDYIAIPKA"
                    "NGYQSLHTSMIGPHGVPVEVQIRTEDMDQMAEMGVAAHWAYKEHGETSTTAQIRAQRWMQ"
                    "SLLELQQSAGSSFEFIESVKSDLFPDEIYVFTPEGRIVELPAGATPVDFAYAVHTDIGHA"
                    "CVGARVDRQPYPLSQPLTSGQTVEIITAPGARPNAAWLNFVVSSKARAKIRQLLKNLKRD"
                    "DSVSLGRRLLNHALGGSRKLNEIPQENIQRELDRMKLATLDDLLAEIGLGNAMSVVVAKN"
                    "LQHGDASIPPATQSHGHLPIKGADGVLITFAKCCRPIPGDPIIAHVSPGKGLVIHHESCR"
                    "NIRGYQKEPEKFMAVEWDKETAQEFITEIKVEMFNHQGALANLTAAINTTTSNIQSLNTE"
                    "EKDGRVYSAFIRLTARDRVHLANIMRKIRVMPDVIKVTRNRN")

yeiB_seq = ("MERNVTLDFVRGVAILGILLLNISAFGLPKAAYLNPAWYGAITPRDAWTWAFLDLIGQVK"
"FLTLFALLFGAGLQMLLPRGRRWIQSRLTLLVLLGFIHGLLFWDGDILLAYGLVGLICWR"
"LVRDAPSVKSLFNTGVMLYLVGLGVLLLLGLISDSQTSRAWTPDASAILYEKYWKLHGGV"
"EAISNRADGVGNSLLALGAQYGWQLAGMMLIGAALMRSGWLKGQFSLRHYRRTGFVLVAI"
"GVTINLPAIALQWQLDWAYRWCAFLLQMPRELSAPFQAIGYASLFYGFWPQLSRFKLVLA"
"IACVGRMALTNYLLQTLICTTLFYHLGLFMHFDRLELLAFVIPVWLANILFSVIWLRYFR"
"QGPVEWLWRQLTLRAAGPAISKTSR")

# List of mutations provided as strings
gene_mutation = {"topA":"H33Y", "spoT":"K662I", "yeiB":"L143I"}

mutations = [[gene_mutation["topA"]], [gene_mutation["spoT"]], [gene_mutation["yeiB"]]]

compare_llr_methods(model, tokenizer, topA_seq, mutations[0])
compare_llr_methods(model_finetuned, tokenizer, topA_seq, mutations[0])
compare_llr_methods(model, tokenizer, spoT_seq, mutations[1])
compare_llr_methods(model_finetuned, tokenizer, spoT_seq, mutations[1])
compare_llr_methods(model, tokenizer, yeiB_seq, mutations[2])
compare_llr_methods(model_finetuned, tokenizer, yeiB_seq, mutations[2])

Mutation H33Y:
  [Function]     LLR: -7.873776, wt: -0.001575, mut: -7.875352
  [HeatmapData]  LLR: -7.873776
  ΔLLR = 0.000000
------------------------------------------------------------
Mutation H33Y:
  [Function]     LLR: 10.498736, wt: -10.498798, mut: -0.000062
  [HeatmapData]  LLR: 10.498736
  ΔLLR = 0.000000
------------------------------------------------------------
Mutation K662I:
  [Function]     LLR: -0.623509, wt: -1.979034, mut: -2.602542
  [HeatmapData]  LLR: -0.623509
  ΔLLR = 0.000000
------------------------------------------------------------
Mutation K662I:
  [Function]     LLR: 8.221296, wt: -8.222016, mut: -0.000720
  [HeatmapData]  LLR: 8.221296
  ΔLLR = 0.000000
------------------------------------------------------------
Mutation L143I:
  [Function]     LLR: -0.479110, wt: -2.057928, mut: -2.537038
  [HeatmapData]  LLR: -0.479110
  ΔLLR = 0.000000
------------------------------------------------------------
Mutation L143I:
  [Function]     LLR: 6.995316, wt: -

In [23]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from transformers import PreTrainedTokenizer

class HeatmapGenerator:
    def __init__(self, model=None, tokenizer=None, model_name=None, is_progen=False):
        """
        Initialize the HeatmapGenerator with either:
        1. A pre-loaded model and tokenizer
        2. A model_name to load from HuggingFace
        
        Args:
            model: A pre-loaded model instance
            tokenizer: A pre-loaded tokenizer
            model_name: HuggingFace model name (used only if model and tokenizer are None)
            is_progen: Set to True when using ProGen models that don't support MLM
        """
        self.is_progen = is_progen
        
        if model is not None and tokenizer is not None:
            self.model = model
            self.tokenizer = tokenizer
        elif model_name is not None:
            if "progen" in model_name.lower():
                self.is_progen = True
                from transformers import AutoModelForCausalLM, AutoTokenizer
                self.tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
                self.model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)
            else:
                from transformers import EsmForMaskedLM, EsmTokenizer
                self.tokenizer = EsmTokenizer.from_pretrained(model_name)
                self.model = EsmForMaskedLM.from_pretrained(model_name)
            if hasattr(self.model, 'eval'):
                self.model.eval()
        else:
            raise ValueError("Either provide a model and tokenizer, or a model_name")

        # device & specials
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)
        self.special_ids = set(self.tokenizer.all_special_ids)
        self.mask_token_id = self.tokenizer.mask_token_id

        print(f"Using model type: {'ProGen' if self.is_progen else 'ESM'} on {self.device}")

    def _get_logits(self, input_ids, attention_mask=None):
        with torch.no_grad():
            input_ids = input_ids.to(self.device)
            if attention_mask is not None:
                attention_mask = attention_mask.to(self.device)
                out = self.model(input_ids, attention_mask=attention_mask)
            else:
                out = self.model(input_ids)
            # extract logits regardless of output type
            return getattr(out, "logits", out[0] if isinstance(out, tuple) else out)

    @staticmethod
    def _map_residue_to_token(input_ids, special_ids, residue_pos):
        """
        Given a 1D tensor of input_ids on CPU and set of special_ids,
        return the token index corresponding to the residue_pos (1-based),
        skipping any special tokens.
        """
        seq_count = 0
        for idx, tid in enumerate(input_ids.tolist()):
            if tid not in special_ids:
                seq_count += 1
                if seq_count == residue_pos:
                    return idx
        raise ValueError(f"Could not map residue {residue_pos} to a token index")

    def llrData(self, protein_sequence, start_pos=1, end_pos=None):
        seq_len = len(protein_sequence)
        if end_pos is None or end_pos > seq_len:
            end_pos = seq_len

        aas = list("ACDEFGHIKLMNPQRSTVWY")
        heatmap = np.zeros((20, end_pos - start_pos + 1), dtype=float)

        # tokenize once if MLM, or nothing if CLM
        if not self.is_progen:
            # Masked LM: we need full sequence tokenization
            enc = self.tokenizer(protein_sequence, return_tensors="pt")
            input_ids_cpu = enc["input_ids"][0]
            attention_cpu = enc.get("attention_mask", None)
            input_ids = enc["input_ids"].to(self.device)
            attention_mask = attention_cpu.to(self.device) if attention_cpu is not None else None
        else:
            # For ProGen, we will retokenize prefixes each time
            special = None  # unused

        for col, pos in enumerate(range(start_pos, end_pos+1)):
            try:
                if not self.is_progen:
                    # ESM path (masked LM)
                    tok_idx = self._map_residue_to_token(input_ids_cpu,
                                                         self.special_ids, pos)
                    wt_id   = input_ids[0, tok_idx].item()
                    masked  = input_ids.clone()
                    masked[0, tok_idx] = self.mask_token_id
                    logits  = self._get_logits(masked, attention_mask)
                    log_probs = torch.log_softmax(logits[0, tok_idx], dim=-1)
                else:
                    # ProGen path (causal LM)
                    # Build prefix up to pos-1
                    prefix = protein_sequence[: pos-1] if pos>1 else ""
                    enc_prefix = self.tokenizer(prefix, return_tensors="pt")
                    ids_cpu = enc_prefix["input_ids"][0]
                    # map pos-> next-token index = len(ids_cpu)-1
                    wt_res = protein_sequence[pos-1]
                    # wildtype token ID
                    wt_id = self.tokenizer.convert_tokens_to_ids(wt_res)
                    # move to device
                    ids = enc_prefix["input_ids"].to(self.device)
                    mask = enc_prefix.get("attention_mask", None)
                    logits_full = self._get_logits(ids, mask)
                    # next-token logits are at last position
                    log_probs = torch.log_softmax(logits_full[0, -1], dim=-1)

                log_wt = log_probs[wt_id].item()
                for i, aa in enumerate(aas):
                    aa_id = self.tokenizer.convert_tokens_to_ids(aa)
                    heatmap[i, col] = (
                        log_probs[aa_id].item() - log_wt
                        if aa_id is not None and aa_id < log_probs.size(0)
                        else 0.0
                    )

            except Exception as e:
                print(f"Error at pos {pos}: {e}")
                heatmap[:, col] = 0.0

        cols = [str(p) for p in range(start_pos, end_pos+1)]
        return pd.DataFrame(heatmap, index=aas, columns=cols)


    def generate_heatmap(self, protein_sequence, start_pos=1, end_pos=None,
                         figsize=(10, 5), cmap="viridis", tick_interval=5, title=None):
        """
        Plots the heatmap of log_prob_mutant - log_prob_wildtype.
        """
        df = self.llrData(protein_sequence, start_pos, end_pos)
        fig, ax = plt.subplots(figsize=figsize)
        cax = ax.imshow(df.values, cmap=cmap, aspect="auto")

        ax.set_xticks(range(df.shape[1]))
        ax.set_xticklabels(
            [col if i % tick_interval == 0 else "" for i, col in enumerate(df.columns)],
            rotation=90, fontsize=8
        )
        ax.set_yticks(range(len(df.index)))
        ax.set_yticklabels(df.index)

        ax.set_xlabel("Position")
        ax.set_ylabel("Amino Acid")
        model_type = "ProGen" if self.is_progen else "ESM"
        ax.set_title(title or f"LLR Heatmap ({model_type})")
        plt.colorbar(cax, ax=ax, label="LLR (mutant vs WT)")
        plt.tight_layout()
        return fig

    def save_heatmap_data(self, protein_sequence, filename, start_pos=1, end_pos=None):
        """
        Compute the LLR data and save it to CSV.
        Returns the DataFrame for further use.
        """
        df = self.llrData(protein_sequence, start_pos, end_pos)
        df.to_csv(filename)
        print(f"Saved LLR data to {filename}")
        return df


In [24]:
pwd

'/home/sdowell/scratch/Thesis'

In [27]:
import torch
from transformers import (
    AutoTokenizer, 
    AutoModelForMaskedLM, 
    AutoModelForCausalLM
)
from peft import PeftModel

# ─── Your existing imports & data ─────────────────────────────────────
# (you already ran these)
from autoamp.evolveFinetune import *  # if needed
from Bio import SeqIO
import numpy as np, pandas as pd

# Sequences & mutations
topA_seq = (
    "MGKALVIVESPAKAKTINKYLGSDYVVKSSVGHIRDLPTSGSAAKKSADSTSTKTAKKPK" 
    "KDERGALVNRMGVDPWHNWEAHYEVLPGKEKVVSELKQLAEKADHIYLATDLDREGEAIA" 
    "WHLREVIGGDDARYSRVVFNEITKNAIRQAFNKPGELNIDRVNAQQARRFMDRVVGYMVS" 
    "PLLWKKIARGLSAGRVQSVAVRLVVEREREIKAFVPEEFWEVDASTTTPSGEALALQVTH" 
    "QNDKPFRPVNKEQTQAAVSLLEKARYSVLEREDKPTTSKPGAPFITSTLQQAASTRLGFG" 
    "VKKTMMMAQRLYEAGYITYMRTDSTNLSQDAVNMVRGYISDNFGKKYLPESPNQYASKEN" 
    "SQEAHEAIRPSDVNVMAESLKDMEADAQKLYQLIWRQFVACQMTPAKYDSTTLTVGAGDF" 
    "RLKARGRILRFDGWTKVMPALRKGDEDRILPAVNKGDALTLVELTPAQHFTKPPARFSEA" 
    "SLVKELEKRGIGRPSTYASIISTIQDRGYVRVENRRFYAEKMGEIVTDRLEENFRELMNY" 
    "DFTAQMENSLDQVANHEAEWKAVLDHFFSDFTQQLDKAEKDPEEGGMRPNQMVLTSIDCP" 
    "TCGRKMGIRTASTGVFLGCSGYALPPKERCKTTINLVPENEVLNVLEGEDAETNALRAKR" 
    "RCPKCGTAMDSYLIDPKRKLHVCGNNPTCDGYEIEEGEFRIKGYDGPIVECEKCGSEMHL" 
    "KMGRFGKYMACTNEECKNTRKILRNGEVAPPKEDPVPLPELPCEKSDAYFVLRDGAAGVF" 
    "LAANTFPKSRETRAPLVEELYRFRDRLPEKLRYLADAPQQDPEGNKTMVRFSRKTKQQYV" 
    "SSEKDGKATGWSAFYVDGKWVEGKK" 
)

spoT_seq = (
        "MYLFESLNQLIQTYLPEDQIKRLRQAYLVARDAHEGQTRSSGEPYITHPVAVACILAEMK"
        "LDYETLMAALLHDVIEDTPATYQDMEQLFGKSVAELVEGVSKLDKLKFRDKKEAQAENFR"
        "KMIMAMVQDIRVILIKLADRTHNMRTLGSLRPDKRRRIARETLEIYSPLAHRLGIHHIKT"
        "ELEELGFEALYPNRYRVIKEVVKAARGNRKEMIQKILSEIEGRLQEAGIPCRVSGREKHL"
        "YSIYCKMVLKEQRFHSIMDIYAFRVIVNDSDTCYRVLGQMHSLYKPRPGRVKDYIAIPKA"
        "NGYQSLHTSMIGPHGVPVEVQIRTEDMDQMAEMGVAAHWAYKEHGETSTTAQIRAQRWMQ"
        "SLLELQQSAGSSFEFIESVKSDLFPDEIYVFTPEGRIVELPAGATPVDFAYAVHTDIGHA"
        "CVGARVDRQPYPLSQPLTSGQTVEIITAPGARPNAAWLNFVVSSKARAKIRQLLKNLKRD"
        "DSVSLGRRLLNHALGGSRKLNEIPQENIQRELDRMKLATLDDLLAEIGLGNAMSVVVAKN"
        "LQHGDASIPPATQSHGHLPIKGADGVLITFAKCCRPIPGDPIIAHVSPGKGLVIHHESCR"
        "NIRGYQKEPEKFMAVEWDKETAQEFITEIKVEMFNHQGALANLTAAINTTTSNIQSLNTE"
        "EKDGRVYSAFIRLTARDRVHLANIMRKIRVMPDVIKVTRNRN")

yeiB_seq = ("MERNVTLDFVRGVAILGILLLNISAFGLPKAAYLNPAWYGAITPRDAWTWAFLDLIGQVK"
        "FLTLFALLFGAGLQMLLPRGRRWIQSRLTLLVLLGFIHGLLFWDGDILLAYGLVGLICWR"
        "LVRDAPSVKSLFNTGVMLYLVGLGVLLLLGLISDSQTSRAWTPDASAILYEKYWKLHGGV"
        "EAISNRADGVGNSLLALGAQYGWQLAGMMLIGAALMRSGWLKGQFSLRHYRRTGFVLVAI"
        "GVTINLPAIALQWQLDWAYRWCAFLLQMPRELSAPFQAIGYASLFYGFWPQLSRFKLVLA"
        "IACVGRMALTNYLLQTLICTTLFYHLGLFMHFDRLELLAFVIPVWLANILFSVIWLRYFR"
        "QGPVEWLWRQLTLRAAGPAISKTSR")

gene_seq = {
    "topA": topA_seq,
    "spoT": spoT_seq,
    "yeiB": yeiB_seq
}
gene_mutation = {"topA":"H33Y", "spoT":"K662I", "yeiB":"L143I"}

# ─── 1. ESM‑2 models (masked LM) ────────────────────────────────────────
esm_name = "facebook/esm2_t30_150M_UR50D"
esm_tok  = AutoTokenizer.from_pretrained(esm_name, use_fast=True)

# pretrained
esm_pre   = AutoModelForMaskedLM.from_pretrained(esm_name)
esm_pre_gen = HeatmapGenerator(model=esm_pre, tokenizer=esm_tok)

# finetuned via PEFT adapter
adapter_checkpoint = "/home/sdowell/scratch/Thesis/BenchmarkingFinetuning/runs/esm_150m_ecoli_finetuning_1/checkpoint-19000"
esm_base   = AutoModelForMaskedLM.from_pretrained(esm_name)
esm_ft     = PeftModel.from_pretrained(esm_base, adapter_checkpoint)
esm_ft_gen = HeatmapGenerator(model=esm_ft, tokenizer=esm_tok)

# ─── 2. ProGen2 models (causal LM) ─────────────────────────────────────
progen_name = "hugohrban/progen2-small"
progen_tok  = AutoTokenizer.from_pretrained(progen_name, trust_remote_code=True)

# pretrained ProGen2
progen_pre   = AutoModelForCausalLM.from_pretrained(progen_name, trust_remote_code=True)
progen_pre_gen = HeatmapGenerator(model=progen_pre, tokenizer=progen_tok, is_progen=True)

# finetuned ProGen2 via PEFT (replace with your path)
progen_adapter_checkpoint = "/home/sdowell/scratch/Thesis/BenchmarkingFinetuning/runs/progen2_151m_ecoli_finetuning_1/checkpoint-11500/"
progen_base = AutoModelForCausalLM.from_pretrained(progen_name, trust_remote_code=True)
progen_ft   = PeftModel.from_pretrained(progen_base, progen_adapter_checkpoint)
progen_ft_gen = HeatmapGenerator(model=progen_ft, tokenizer=progen_tok, is_progen=True)

# ── Loop & save CSVs across the full sequence ─────────────────────────
for model_label, gen in [
    ("esm_pretrained",    esm_pre_gen),
    ("esm_finetuned",     esm_ft_gen),
    ("progen_pretrained", progen_pre_gen),
    ("progen_finetuned",  progen_ft_gen),
]:
    for gene, seq in gene_seq.items():
        filename = f"{gene}_{model_label}_full_sequence.csv"
        print(f"Saving full‐sequence LLRs to {filename} ...")
        # omit start_pos/end_pos → defaults to entire sequence
        df = gen.save_heatmap_data(
            protein_sequence=seq,
            filename=filename
        )
        # (optional) inspect the head
        print(df.head(), "\n")


Using model type: ESM on cuda
Using model type: ESM on cuda
Using model type: ProGen on cuda
Using model type: ProGen on cuda
Saving full‐sequence LLRs to topA_esm_pretrained_full_sequence.csv ...
Saved LLR data to topA_esm_pretrained_full_sequence.csv
           1         2         3         4          5          6          7  \
A  -7.860852  0.898954 -4.338935  0.000000  -6.224969  -4.939164  -9.669122   
C -10.719876 -5.129097 -7.209413 -3.161048  -7.122352  -3.875269 -10.084961   
D  -8.410981 -1.875968 -4.203128 -3.031852 -13.915084 -13.362583 -14.192044   
E  -8.406668 -1.254799 -3.644945 -1.901605 -12.897623 -12.520166 -13.429348   
F  -9.296198 -4.304859 -6.416578 -4.530185  -6.975884  -3.493910  -6.829610   

           8          9         10  ...       856       857       858  \
A  -1.537119 -11.029301  -4.220697  ... -0.929391 -2.887781 -5.453708   
C  -3.873991 -12.514373  -8.771953  ... -4.318420 -6.371966 -7.500591   
D -11.427444  -8.036858 -10.795480  ... -2.040178  0.